In [2]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Insert URL for car make and model on auto-data.net
url= "https://www.auto-data.net/en/bmw-3-series-model-953"
gen_url= 'https://www.auto-data.net'

In [3]:
# Test url response
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [4]:
data = response.text
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# Create table and dictionary of all the generations of the vehicle
table = soup.find('table', {'class': 'generr'})
rows = table.findAll('th')
links =[]
models =[]
model_link = {}
for th in rows:
    cols = th.findAll('a')
    for a in range(len(cols)):
        link = cols[a].get('href')
        model = cols[a].get('title')
        links.append(link)
        models.append(model)
        model_link[model] =link

In [6]:
# Iterate though each generation and get link to differnt models avaible in that generation
submodel={}
for value in model_link:
    url_loop = gen_url + model_link[value]
    response_loop = requests.get(url_loop, headers=headers)
    data2 = response_loop.text
    soup2 = BeautifulSoup(data2, 'html.parser')

    table2 = soup2.find('table', {'class': 'carlist'})
    rows = table2.findAll('th')
    for th in rows:
        cols = th.findAll('a')
        for a in range(len(cols)):
            link = cols[a].get('href')
            model = cols[a].get('title')
            submodel[model] =link
    


In [7]:
submodel

{'2022 BMW 3 Series Sedan (G20 LCI, facelift 2022) M340i (374 Hp) Mild Hybrid xDrive Steptronic - Technical Specs, Fuel consumption, Dimensions': '/en/bmw-3-series-sedan-g20-lci-facelift-2022-m340i-374hp-mild-hybrid-xdrive-steptronic-45839',
 '2022 BMW 3 Series Sedan (G20 LCI, facelift 2022) M340d (340 Hp) Mild Hybrid xDrive Steptronic - Technical Specs, Fuel consumption, Dimensions': '/en/bmw-3-series-sedan-g20-lci-facelift-2022-m340d-340hp-mild-hybrid-xdrive-steptronic-45824',
 '2022 BMW 3 Series Sedan (G20 LCI, facelift 2022) 330i (245 Hp) xDrive Steptronic - Technical Specs, Fuel consumption, Dimensions': '/en/bmw-3-series-sedan-g20-lci-facelift-2022-330i-245hp-xdrive-steptronic-45815',
 '2022 BMW 3 Series Sedan (G20 LCI, facelift 2022) 330i (245 Hp) Steptronic - Technical Specs, Fuel consumption, Dimensions': '/en/bmw-3-series-sedan-g20-lci-facelift-2022-330i-245hp-steptronic-45814',
 '2022 BMW 3 Series Sedan (G20 LCI, facelift 2022) 330e (292 Hp) xDrive Steptronic - Technical Spe

In [8]:
# Create DataFrame to hold information
df = pd.DataFrame()


In [9]:
df

""


In [10]:
# Test data ETL on one car model
url_loop = gen_url + list(submodel.values())[0]
response_loop = requests.get(url_loop, headers=headers)
data2 = response_loop.text
soup2 = BeautifulSoup(data2, 'html.parser')

table = soup2.find('table', {'class': 'cardetailsout'})
tbody = table.findAll('tr')
car_table = {}
# car_table[tbody[1].find('th').text] = tbody[1].find('td').text
for tr in range(1,len(tbody)):
     try:
          car_table[tbody[tr].find('th').text] = tbody[tr].find('td').text
     except AttributeError:
          continue

In [11]:
car_table

{'Brand': 'BMW',
 'Model ': '3 Series',
 'Generation  ': '3 Series Sedan (G20 LCI, facelift 2022)',
 'Modification (Engine) ': 'M340i (374 Hp) Mild Hybrid xDrive Steptronic ',
 'Start of production ': 'July, 2022 year ',
 'Powertrain Architecture ': 'MHEV (Mild Hybrid Electric Vehicle, power-assist hybrid, battery-assisted hybrid vehicles, BAHV) ',
 'Body type': 'Sedan ',
 'Seats ': '5 ',
 'Doors ': '4 ',
 'Combined fuel consumption (WLTP) ': '7.8-8.5 l/100 km\r\n\t\t\t\t\t\t\t30.16 - 27.67 US mpg36.22 - 33.23 UK mpg12.82 - 11.76 km/l',
 'CO2 emissions (WLTP)': '177-192 g/km ',
 'Fuel Type ': 'Petrol (Gasoline) ',
 'Acceleration 0 - 100 km/h': '4.4 sec ',
 'Acceleration 0 - 62 mph': '4.4 sec ',
 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '4.2 sec ',
 'Maximum speed ': '250 km/h\r\n\t\t\t\t\t\t155.34 mph',
 'Emission standard ': 'Euro 6d ',
 'Weight-to-power ratio ': '4.6 kg/Hp, 216.8 Hp/tonne ',
 'Weight-to-torque ratio ': '3.5 kg/Nm, 289.9 Nm/tonne ',
 'Battery voltage 

In [34]:
# Clear DataFrame to hold information
df = pd.DataFrame()

In [35]:
# Iterate through each generations models and pull information to put into pandas dataframe

for value in submodel:
   url_loop = gen_url + submodel[value]
   response_loop = requests.get(url_loop, headers=headers)
   data2 = response_loop.text
   soup2 = BeautifulSoup(data2, 'html.parser')

   table = soup2.find('table', {'class': 'cardetailsout'})
   tbody = table.findAll('tr')
   car_table = {}
   #currently adding each loop to a new row rather than adding to th exisiting row for the gemeration model
   for tr in range(len(tbody)):
      try:
         car_table[tbody[tr].find('th').text] = tbody[tr].find('td').text
      except AttributeError:
         continue
   # print(car_table)
   df = df.append(car_table, ignore_index=True)

{'Brand': 'BMW', 'Model ': '3 Series', 'Generation  ': '3 Series Sedan (G20 LCI, facelift 2022)', 'Modification (Engine) ': 'M340i (374 Hp) Mild Hybrid xDrive Steptronic ', 'Start of production ': 'July, 2022 year ', 'Powertrain Architecture ': 'MHEV (Mild Hybrid Electric Vehicle, power-assist hybrid, battery-assisted hybrid vehicles, BAHV) ', 'Body type': 'Sedan ', 'Seats ': '5 ', 'Doors ': '4 ', 'Combined fuel consumption (WLTP) ': '7.8-8.5 l/100 km\r\n\t\t\t\t\t\t\t30.16 - 27.67 US mpg36.22 - 33.23 UK mpg12.82 - 11.76 km/l', 'CO2 emissions (WLTP)': '177-192 g/km ', 'Fuel Type ': 'Petrol (Gasoline) ', 'Acceleration 0 - 100 km/h': '4.4 sec ', 'Acceleration 0 - 62 mph': '4.4 sec ', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '4.2 sec ', 'Maximum speed ': '250 km/h\r\n\t\t\t\t\t\t155.34 mph', 'Emission standard ': 'Euro 6d ', 'Weight-to-power ratio ': '4.6 kg/Hp, 216.8 Hp/tonne ', 'Weight-to-torque ratio ': '3.5 kg/Nm, 289.9 Nm/tonne ', 'Battery voltage ': '48 V ', 'Batter

C:\Users\nizar\AppData\Local\Temp\ipykernel_1884\4289994755.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(car_table, ignore_index=True)


{'Brand': 'BMW', 'Model ': '3 Series', 'Generation  ': '3 Series Sedan (G20 LCI, facelift 2022)', 'Modification (Engine) ': 'M340d (340 Hp) Mild Hybrid xDrive Steptronic ', 'Start of production ': 'July, 2022 year ', 'Powertrain Architecture ': 'MHEV (Mild Hybrid Electric Vehicle, power-assist hybrid, battery-assisted hybrid vehicles, BAHV) ', 'Body type': 'Sedan ', 'Seats ': '5 ', 'Doors ': '4 ', 'Combined fuel consumption (WLTP) ': '5.9-6.4 l/100 km\r\n\t\t\t\t\t\t\t39.87 - 36.75 US mpg47.88 - 44.14 UK mpg16.95 - 15.63 km/l', 'CO2 emissions (WLTP)': '154-168 g/km ', 'Fuel Type ': 'Diesel ', 'Acceleration 0 - 100 km/h': '4.6 sec ', 'Acceleration 0 - 62 mph': '4.6 sec ', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '4.4 sec ', 'Maximum speed ': '250 km/h\r\n\t\t\t\t\t\t155.34 mph', 'Emission standard ': 'Euro 6d ', 'Weight-to-power ratio ': '5.3 kg/Hp, 188.9 Hp/tonne ', 'Weight-to-torque ratio ': '2.6 kg/Nm, 388.9 Nm/tonne ', 'Battery voltage ': '48 V ', 'Battery technolog

C:\Users\nizar\AppData\Local\Temp\ipykernel_1884\4289994755.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(car_table, ignore_index=True)


{'Brand': 'BMW', 'Model ': '3 Series', 'Generation  ': '3 Series Sedan (G20 LCI, facelift 2022)', 'Modification (Engine) ': '330i (245 Hp) xDrive Steptronic ', 'Start of production ': 'July, 2022 year ', 'Powertrain Architecture ': 'Internal Combustion engine ', 'Body type': 'Sedan ', 'Seats ': '5 ', 'Doors ': '4 ', 'Combined fuel consumption (WLTP) ': '7-7.8 l/100 km\r\n\t\t\t\t\t\t\t33.6 - 30.16 US mpg40.35 - 36.22 UK mpg14.29 - 12.82 km/l', 'CO2 emissions (WLTP)': '158-176 g/km ', 'Fuel Type ': 'Petrol (Gasoline) ', 'Acceleration 0 - 100 km/h': '5.7 sec ', 'Acceleration 0 - 62 mph': '5.7 sec ', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '5.4 sec ', 'Maximum speed ': '250 km/h\r\n\t\t\t\t\t\t155.34 mph', 'Emission standard ': 'Euro 6d ', 'Weight-to-power ratio ': '6.5 kg/Hp, 153.6 Hp/tonne ', 'Weight-to-torque ratio ': '4 kg/Nm, 250.8 Nm/tonne ', 'Power ': '245 Hp @ 4500-6500 rpm. ', 'Power per litre ': '122.6 Hp/l ', 'Torque ': '400 Nm @ 1600-4000 rpm.\r\n\t\t\t\t\t\t

...

C:\Users\nizar\AppData\Local\Temp\ipykernel_1884\4289994755.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(car_table, ignore_index=True)


{'Brand': 'BMW', 'Model ': '3 Series', 'Generation  ': '3 Series (E21)', 'Modification (Engine) ': '316 (90 Hp) ', 'Start of production ': '1975 year ', 'End of production ': '1980 year ', 'Powertrain Architecture ': 'Internal Combustion engine ', 'Body type': 'Sedan ', 'Seats ': '5 ', 'Doors ': '2 ', 'Fuel consumption (economy) - urban ': '11.1 l/100 km\r\n\t\t\t\t\t\t\t21.19 US mpg25.45 UK mpg9.01 km/l', 'Fuel consumption (economy) - extra urban': '6.2 l/100 km\r\n\t\t\t\t\t\t\t37.94 US mpg45.56 UK mpg16.13 km/l', 'Fuel consumption (economy) - combined': '8.3 l/100 km\r\n\t\t\t\t\t\t\t28.34 US mpg34.03 UK mpg12.05 km/l', 'Fuel Type ': 'Petrol (Gasoline) ', 'Acceleration 0 - 100 km/h': '12.5 sec ', 'Acceleration 0 - 62 mph': '12.5 sec ', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '11.9 sec ', 'Maximum speed ': '167 km/h\r\n\t\t\t\t\t\t103.77 mph', 'Weight-to-power ratio ': '11.3 kg/Hp, 88.2 Hp/tonne ', 'Power ': '90 Hp ', 'Power per litre ': '57.2 Hp/l ', 'Engine layout

C:\Users\nizar\AppData\Local\Temp\ipykernel_1884\4289994755.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(car_table, ignore_index=True)


{'Brand': 'BMW', 'Model ': '3 Series', 'Generation  ': '3 Series (E21)', 'Modification (Engine) ': '315 (75 Hp) ', 'Start of production ': '1981 year ', 'End of production ': '1984 year ', 'Powertrain Architecture ': 'Internal Combustion engine ', 'Body type': 'Sedan ', 'Seats ': '5 ', 'Doors ': '2 ', 'Fuel consumption (economy) - urban ': '10.5 l/100 km\r\n\t\t\t\t\t\t\t22.4 US mpg26.9 UK mpg9.52 km/l', 'Fuel consumption (economy) - extra urban': '6.8 l/100 km\r\n\t\t\t\t\t\t\t34.59 US mpg41.54 UK mpg14.71 km/l', 'Fuel Type ': 'Petrol (Gasoline) ', 'Acceleration 0 - 100 km/h': '14.8 sec ', 'Acceleration 0 - 62 mph': '14.8 sec ', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ': '14.1 sec ', 'Maximum speed ': '154 km/h\r\n\t\t\t\t\t\t95.69 mph', 'Weight-to-power ratio ': '13.5 kg/Hp, 74.3 Hp/tonne ', 'Weight-to-torque ratio ': '9.2 kg/Nm, 108.9 Nm/tonne ', 'Power ': '75 Hp @ 5800 rpm. ', 'Power per litre ': '47.7 Hp/l ', 'Torque ': '110 Nm @ 3200 rpm.\r\n\t\t\t\t\t\t81.13 lb.-f

C:\Users\nizar\AppData\Local\Temp\ipykernel_1884\4289994755.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(car_table, ignore_index=True)


In [36]:
#save raw data to csv
df.to_csv('BMW_3_Series.csv')

In [261]:
#read csv
df = pd.read_csv('BMW_3_Series.csv')

In [262]:
df.head(20)

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,CO2 emissions (NEDC),Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - extra urban (NEDC),Fuel consumption (economy) - urban (NEDC),"All-electric range (NEDC, WLTP equivalent)",All-electric range (NEDC),Average Energy consumption (NEDC),Acceleration 0 - 60 mph,Width with mirrors folded,Maximum engine speed
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330d (286 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330d (286 Hp) Mild Hybrid Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",320i (184 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",320i (184 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
df.shape

(738, 107)

In [264]:
columns = list(df.columns)

In [265]:
for i in columns:
    print(i)

Unnamed: 0
Brand
Model 
Generation  
Modification (Engine) 
Start of production 
Powertrain Architecture 
Body type
Seats 
Doors 
Combined fuel consumption (WLTP) 
CO2 emissions (WLTP)
Fuel Type 
Acceleration 0 - 100 km/h
Acceleration 0 - 62 mph
Acceleration 0 - 60 mph (Calculated by Auto-Data.net) 
Maximum speed 
Emission standard 
Weight-to-power ratio 
Weight-to-torque ratio 
Battery voltage 
Battery location 
Electric motor power 
Power 
Power per litre 
Torque 
Engine layout 
Engine Model/Code 
Engine displacement 
Number of cylinders 
Engine configuration 
Cylinder Bore 
Piston Stroke 
Compression ratio 
Number of valves per cylinder 
Fuel injection system 
Engine aspiration 
Valvetrain  
Engine oil capacity 
Engine oil specification 
Engine systems
Kerb Weight 
Max. weight 
Max load
Trunk (boot) space - minimum 
Fuel tank capacity 
Max. roof load 
Permitted trailer load with brakes (12%) 
Permitted trailer load without brakes 
Permitted towbar download 
Length 
Width 
Height 
Wh

In [266]:
# Data Cleaning
# merge columns
    # CO2 emissions (WLTP), CO2 emissions, CO2 emissions (NEDC, WLTP equivalent), CO2 emissions (NEDC)
    # Combined fuel consumption (WLTP), Fuel consumption (economy) - combined, Fuel consumption (economy) - combined (NEDC, WLTP equivalent), Fuel consumption (economy) - combined (NEDC)
# Drop
#('Acceleration 0 - 62 mph', 'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ', 'Engine layout ', 'Engine Model/Code ', 'Engine configuration ','Cylinder Bore ','Piston Stroke ', 'Compression ratio ','Number of valves per cylinder ','Fuel injection system ,Valvetrain ','Engine oil capacity ',
# 'Engine oil specification ',
# 'Engine systems',
# 'Max. weight ',
# 'Max load',
# 'Trunk (boot) space - minimum ',
# 'Fuel tank capacity ',
# 'Max. roof load ',
# 'Permitted trailer load with brakes (12%) ',
# 'Permitted trailer load without brakes ',
# 'Permitted towbar download ',
# 'Length ',
# 'Width ',
# 'Height ',
# 'Wheelbase ',
# 'Front track ',
# 'Rear (Back) track ',
# 'Ride height (ground clearance)',
# 'Minimum turning circle (turning diameter) ',
# 'Drivetrain Architecture ',
# 'Number of gears and type of gearbox ',
# 'Front suspension ',
# 'Rear suspension ',
# 'Front brakes',
# 'Rear brakes',
# 'Assisting systems',
# 'Steering type ',
# 'Power steering ',
# 'Tires size',
# 'Wheel rims size',
# 'Trunk (boot) space - maximum ',
# 'Coolant ',
# 'End of production ',
# 'Fuel consumption (economy) - extra urban',
# 'Fuel consumption (economy) - urban ',
# 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
# 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent) ',
# 'Front overhang ',
# 'Rear overhang ',
# 'Width including mirrors ',
# 'Permitted trailer load with brakes (8%) '
# 'Fuel consumption (economy) - extra urban (NEDC)',
# 'Fuel consumption (economy) - urban (NEDC) ',
# 'Width with mirrors folded ',
# 'Maximum engine speed '
# 'Average Energy consumption (NEDC) ')


In [267]:
drop = ['Acceleration 0 - 62 mph',
         'Acceleration 0 - 60 mph (Calculated by Auto-Data.net) ',
           'Engine layout ',
             'Engine Model/Code ',
               'Engine configuration ',
               'Cylinder Bore ',
               'Piston Stroke ',
                 'Compression ratio ',
                 'Number of valves per cylinder ',
                 'Fuel injection system ',
                 'Valvetrain  ',
                 'Engine oil capacity ',
'Engine oil specification ',
'Engine systems',
'Max. weight ',
'Max load',
'Trunk (boot) space - minimum ',
'Fuel tank capacity ',
'Max. roof load ',
'Permitted trailer load with brakes (12%) ',
'Permitted trailer load without brakes ',
'Permitted towbar download ',
'Length ',
'Width ',
'Height ',
'Wheelbase ',
'Front track ',
'Rear (Back) track ',
'Ride height (ground clearance) ',
'Minimum turning circle (turning diameter) ',
'Drivetrain Architecture ',
'Number of gears and type of gearbox ',
'Front suspension ',
'Rear suspension ',
'Front brakes',
'Rear brakes',
'Assisting systems',
'Steering type ',
'Power steering ',
'Tires size',
'Wheel rims size',
'Trunk (boot) space - maximum ',
'Coolant ',
'End of production ',
'Fuel consumption (economy) - extra urban',
'Fuel consumption (economy) - urban ',
'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
'Fuel consumption (economy) - urban (NEDC, WLTP equivalent) ',
'Front overhang ',
'Rear overhang ',
'Width including mirrors ',
'Permitted trailer load with brakes (8%) ',
'Fuel consumption (economy) - extra urban (NEDC)',
'Fuel consumption (economy) - urban (NEDC) ',
'Width with mirrors folded ',
'Maximum engine speed ',
'Average Energy consumption (NEDC) ']

In [268]:
df.head()

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,CO2 emissions (NEDC),Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - extra urban (NEDC),Fuel consumption (economy) - urban (NEDC),"All-electric range (NEDC, WLTP equivalent)",All-electric range (NEDC),Average Energy consumption (NEDC),Acceleration 0 - 60 mph,Width with mirrors folded,Maximum engine speed
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
df_dropped = df.drop(columns=drop)

In [270]:
df_dropped.head()

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,Fuel consumption (economy) - combined,CO2 emissions,"CO2 emissions (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (NEDC, WLTP equivalent)","Average Energy consumption (NEDC, WLTP equivalent)",CO2 emissions (NEDC),Fuel consumption (economy) - combined (NEDC),"All-electric range (NEDC, WLTP equivalent)",All-electric range (NEDC),Acceleration 0 - 60 mph
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [271]:
# Merge and Clean CO2 emissions data
df_CO2 = pd.DataFrame({'CO2 emissions (WLTP)': df_dropped['CO2 emissions (WLTP)'], 'CO2 emissions':df_dropped['CO2 emissions'], 'CO2 emissions (NEDC, WLTP equivalent)':df_dropped['CO2 emissions (NEDC, WLTP equivalent)'], 'CO2 emissions (NEDC)':df_dropped['CO2 emissions (NEDC)']})

In [272]:
df_CO2

,CO2 emissions (WLTP),CO2 emissions,"CO2 emissions (NEDC, WLTP equivalent)",CO2 emissions (NEDC)
0,177-192 g/km,NaN,NaN,NaN
1,154-168 g/km,NaN,NaN,NaN
2,158-176 g/km,NaN,NaN,NaN
3,148-166 g/km,NaN,NaN,NaN
4,33-45 g/km,NaN,NaN,NaN
...,...,...,...,...
733,NaN,NaN,NaN,NaN
734,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN
736,NaN,NaN,NaN,NaN


In [273]:
df_CO2.isna().sum()

CO2 emissions (WLTP)                     701
CO2 emissions                            411
CO2 emissions (NEDC, WLTP equivalent)    721
CO2 emissions (NEDC)                     701
dtype: int64

In [274]:
# Create new column that will fill with data from each other column
df_CO2['CO2 Emissions (g/km)'] = df_CO2['CO2 emissions (WLTP)']

In [275]:
df_CO2

,CO2 emissions (WLTP),CO2 emissions,"CO2 emissions (NEDC, WLTP equivalent)",CO2 emissions (NEDC),CO2 Emissions (g/km)
0,177-192 g/km,NaN,NaN,NaN,177-192 g/km
1,154-168 g/km,NaN,NaN,NaN,154-168 g/km
2,158-176 g/km,NaN,NaN,NaN,158-176 g/km
3,148-166 g/km,NaN,NaN,NaN,148-166 g/km
4,33-45 g/km,NaN,NaN,NaN,33-45 g/km
...,...,...,...,...,...
733,NaN,NaN,NaN,NaN,NaN
734,NaN,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN,NaN
736,NaN,NaN,NaN,NaN,NaN


In [276]:
df_CO2['CO2 Emissions (g/km)'] = df_CO2['CO2 Emissions (g/km)'].fillna(df_CO2['CO2 emissions'])


In [277]:
df_CO2.isna().sum()

CO2 emissions (WLTP)                     701
CO2 emissions                            411
CO2 emissions (NEDC, WLTP equivalent)    721
CO2 emissions (NEDC)                     701
CO2 Emissions (g/km)                     374
dtype: int64

In [278]:
df_CO2['CO2 Emissions (g/km)'] = df_CO2['CO2 Emissions (g/km)'].fillna(df_CO2['CO2 emissions (NEDC, WLTP equivalent)'])

In [279]:
df_CO2.isna().sum()

CO2 emissions (WLTP)                     701
CO2 emissions                            411
CO2 emissions (NEDC, WLTP equivalent)    721
CO2 emissions (NEDC)                     701
CO2 Emissions (g/km)                     363
dtype: int64

In [280]:
df_CO2['CO2 Emissions (g/km)'] = df_CO2['CO2 Emissions (g/km)'].fillna(df_CO2['CO2 emissions (NEDC)'])

In [281]:
df_CO2.isna().sum()

CO2 emissions (WLTP)                     701
CO2 emissions                            411
CO2 emissions (NEDC, WLTP equivalent)    721
CO2 emissions (NEDC)                     701
CO2 Emissions (g/km)                     327
dtype: int64

In [282]:
df_CO2 = df_CO2.drop(columns=['CO2 emissions (WLTP)',	'CO2 emissions',	'CO2 emissions (NEDC, WLTP equivalent)',	'CO2 emissions (NEDC)'])

In [283]:
df_CO2['CO2 Emissions (g/km)'].unique()

array(['177-192 g/km ', '154-168 g/km ', '158-176 g/km ', '148-166 g/km ',
       '33-45 g/km ', '30-40 g/km ', '141-158 g/km ', '135-152 g/km ',
       '153-171 g/km ', '145-162 g/km ', '29-40 g/km ', '135-153 g/km ',
       '127-145 g/km ', '182-197 g/km ', '157-173 g/km ', '164-182 g/km ',
       '154-172 g/km ', '35-48 g/km ', '31-42 g/km ', '148-165 g/km ',
       '140-157 g/km ', '151-168 g/km ', '34-48 g/km ', '31-43 g/km ',
       '139-157 g/km ', '132-150 g/km ', '132-149 g/km ', nan,
       '163-172 g/km ', '143-153 g/km ', '143-151 g/km ', '136-146 g/km ',
       '127-136 g/km ', '119-128 g/km ', '140-146 g/km ', '136-144 g/km ',
       '130-138 g/km ', '35-49 g/km ', '32-44 g/km ', '121-129 g/km ',
       '115-125 g/km ', '117-119 g/km ', '109-112 g/km ', '119-125 g/km ',
       '130-140 g/km ', '114-123 g/km ', '118-127 g/km ', '160-168 g/km ',
       '139-149 g/km ', '141-147 g/km ', '132-139 g/km ', '37-43 g/km ',
       '123-132 g/km ', '116-125 g/km ', '134-140 g/km ',

In [284]:
df_CO2.dtypes

CO2 Emissions (g/km)    object
dtype: object

In [285]:
# remove unwanted characters in the data then split range value and find mean of the range
df_CO2['CO2 Emissions (g/km)'] = df_CO2['CO2 Emissions (g/km)'].str.replace('\D', ' ', regex=True)

In [286]:
df_CO2

,CO2 Emissions (g/km)
0,177 192
1,154 168
2,158 176
3,148 166
4,33 45
...,...
733,NaN
734,NaN
735,NaN
736,NaN


In [287]:
df_CO2[['cc','c2']] = df_CO2["CO2 Emissions (g/km)"].str.split(" ", 1, expand=True)

In [288]:
df_CO2

,CO2 Emissions (g/km),cc,c2
0,177 192,177,192
1,154 168,154,168
2,158 176,158,176
3,148 166,148,166
4,33 45,33,45
...,...,...,...
733,NaN,NaN,NaN
734,NaN,NaN,NaN
735,NaN,NaN,NaN
736,NaN,NaN,NaN


In [289]:
df_CO2['c2'] = df_CO2['c2'].str.strip()

In [290]:
df_CO2 = df_CO2.replace(r'^\s*$', np.nan, regex=True)

In [291]:
df_CO2['c2'] = df_CO2['c2'].fillna(df_CO2['cc'])

In [292]:
df_CO2['c2'][157]

'187'

In [293]:
df_CO2['cc'] = pd.to_numeric(df_CO2['cc'])

In [294]:
df_CO2['c2'] = pd.to_numeric(df_CO2['c2'])

In [295]:
df_CO2.dtypes

CO2 Emissions (g/km)     object
cc                      float64
c2                      float64
dtype: object

In [296]:
df_CO2["CO2 Emissions (g/km)"] = df_CO2.mean(axis=1)

C:\Users\nizar\AppData\Local\Temp\ipykernel_26288\3710598989.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_CO2["CO2 Emissions (g/km)"] = df_CO2.mean(axis=1)


In [297]:
df_CO2

,CO2 Emissions (g/km),cc,c2
0,184.5,177.0,192.0
1,161.0,154.0,168.0
2,167.0,158.0,176.0
3,157.0,148.0,166.0
4,39.0,33.0,45.0
...,...,...,...
733,NaN,NaN,NaN
734,NaN,NaN,NaN
735,NaN,NaN,NaN
736,NaN,NaN,NaN


In [298]:
df_CO2 = df_CO2.drop(columns=['cc','c2'])

In [299]:
df_CO2

,CO2 Emissions (g/km)
0,184.5
1,161.0
2,167.0
3,157.0
4,39.0
...,...
733,NaN
734,NaN
735,NaN
736,NaN


In [300]:
# add CO2 data to dropped dataframe and drop old CO2 data
df_dropped['CO2 Emissions (g/km)'] = df_CO2['CO2 Emissions (g/km)']

In [301]:
df_dropped = df_dropped.drop(columns=['CO2 emissions (WLTP)',	'CO2 emissions',	'CO2 emissions (NEDC, WLTP equivalent)',	'CO2 emissions (NEDC)'])

In [302]:
df_dropped

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,System power,System torque,Fuel consumption (economy) - combined,"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)","Average Energy consumption (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (NEDC),"All-electric range (NEDC, WLTP equivalent)",All-electric range (NEDC),Acceleration 0 - 60 mph,CO2 Emissions (g/km)
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.5
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,292 Hp,420 Nm\r\n\t\t\t\t309.78 lb.-ft.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,733,BMW,3 Series,3 Series (E21),318i (105 Hp) Automatic,1979 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,734,BMW,3 Series,3 Series (E21),318i (105 Hp),1979 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,8.4 l/100 km\r\n\t\t\t\t\t\t\t28 US mpg33.63 U...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,735,BMW,3 Series,3 Series (E21),318 (98 Hp),1975 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736,736,BMW,3 Series,3 Series (E21),316 (90 Hp),1975 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,8.3 l/100 km\r\n\t\t\t\t\t\t\t28.34 US mpg34.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
# do same for fuel consumption

df_consumption = pd.DataFrame({'Combined fuel consumption (WLTP)': df_dropped['Combined fuel consumption (WLTP) '], 'Fuel consumption (economy) - combined' : df_dropped['Fuel consumption (economy) - combined'],
                                 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)' : df_dropped['Fuel consumption (economy) - combined (NEDC, WLTP equivalent)'], 'Fuel consumption (economy) - combined (NEDC)' :df_dropped['Fuel consumption (economy) - combined (NEDC)']})

In [304]:
df_consumption

,Combined fuel consumption (WLTP),Fuel consumption (economy) - combined,"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (NEDC)
0,7.8-8.5 l/100 km\r\n\t\t\t\t\t\t\t30.16 - 27.6...,NaN,NaN,NaN
1,5.9-6.4 l/100 km\r\n\t\t\t\t\t\t\t39.87 - 36.7...,NaN,NaN,NaN
2,7-7.8 l/100 km\r\n\t\t\t\t\t\t\t33.6 - 30.16 U...,NaN,NaN,NaN
3,6.6-7.3 l/100 km\r\n\t\t\t\t\t\t\t35.64 - 32.2...,NaN,NaN,NaN
4,1.5-2 l/100 km\r\n\t\t\t\t\t\t\t156.81 - 117.6...,NaN,NaN,NaN
...,...,...,...,...
733,NaN,NaN,NaN,NaN
734,NaN,8.4 l/100 km\r\n\t\t\t\t\t\t\t28 US mpg33.63 U...,NaN,NaN
735,NaN,NaN,NaN,NaN
736,NaN,8.3 l/100 km\r\n\t\t\t\t\t\t\t28.34 US mpg34.0...,NaN,NaN


In [305]:
df_consumption.dtypes

Combined fuel consumption (WLTP)                                 object
Fuel consumption (economy) - combined                            object
Fuel consumption (economy) - combined (NEDC, WLTP equivalent)    object
Fuel consumption (economy) - combined (NEDC)                     object
dtype: object

In [306]:
df_consumption['Combined fuel consumption (WLTP)'] = df_consumption['Combined fuel consumption (WLTP)'].str.split('l').str[0]
df_consumption['Fuel consumption (economy) - combined'] = df_consumption['Fuel consumption (economy) - combined'].str.split('l').str[0]
df_consumption['Fuel consumption (economy) - combined (NEDC, WLTP equivalent)'] = df_consumption['Fuel consumption (economy) - combined (NEDC, WLTP equivalent)'].str.split('l').str[0]
df_consumption['Fuel consumption (economy) - combined (NEDC)'] = df_consumption['Fuel consumption (economy) - combined (NEDC)'].str.split('l').str[0]

In [307]:
df_consumption

,Combined fuel consumption (WLTP),Fuel consumption (economy) - combined,"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (NEDC)
0,7.8-8.5,NaN,NaN,NaN
1,5.9-6.4,NaN,NaN,NaN
2,7-7.8,NaN,NaN,NaN
3,6.6-7.3,NaN,NaN,NaN
4,1.5-2,NaN,NaN,NaN
...,...,...,...,...
733,NaN,NaN,NaN,NaN
734,NaN,8.4,NaN,NaN
735,NaN,NaN,NaN,NaN
736,NaN,8.3,NaN,NaN


In [308]:
for col in df_consumption:
    print(df_consumption[col].unique())

['7.8-8.5 ' '5.9-6.4 ' '7-7.8 ' '6.6-7.3 ' '1.5-2 ' '1.3-1.8 ' '5.4-6 '
 '5.1-5.8 ' '6.8-7.6 ' '6.4-7.2 ' '4.8-5.5 ' '8.1-8.8 ' '6-6.6 '
 '7.3-8.1 ' '1.5-2.1 ' '1.4-1.9 ' '5.6-6.3 ' '5.3-6 ' '6.7-7.5 ' '5-5.7 '
 nan '1.6-2.1 ' '1.5-2.2 ' '6.3-7.1 ']
[nan '6.8 ' '6 ' '7.2-7.6 ' '6.3-6.6 ' '6.0-6.4 ' '5.4-5.6 ' '5.2-5.5 '
 '5.7-6.1 ' '4.6-4.9 ' '4.4-4.8 ' '4.6-4.8 ' '4.4-4.7 ' '4.5-4.8 ' '9.4 '
 '7.0-7.4 ' '6.2-6.5 ' '5.8-6.1 ' '8.4 ' '7.8 ' '5.1-5.4 ' '5.0-5.2 '
 '5.9-6.3 ' '5.5-5.9 ' '4.2-4.5 ' '4.3-4.6 ' '7.7-7.4 ' '7.3-7.0 '
 '5.7-5.6 ' '6.7-6.3 ' '6.2-5.9 ' '5.6-5.4 ' '5.3-5.1 ' '5.0-4.7 '
 '6.6-6.3 ' '7.1-6.7 ' '6.1-5.8 ' '6.4-6.1 ' '4.8-4.5 ' '5.1-4.8 '
 '4.6-4.3 ' '4.8-4.6 ' '4.5-4.3 ' '4.7-4.5 ' '7.6-7.2 ' '7.0-6.8 '
 '6.5-6.2 ' '6.7-6.4 ' '5.5-5.4 ' '4.7-5.0 ' '4.9-5.2 ' '4.9-4.8 '
 '5.2-5.1 ' '6.5-6.1 ' '7.3-6.8 ' '5.9-5.5 ' '6.3-5.9 ' '4.9-4.5 '
 '4.5-4.1 ' '4.7-4.3 ' '4.4-4.0 ' '5.8-5.3 ' '5.9-5.4 ' '5.1-4.7 '
 '4.6-4.1 ' '7.3-6.9 ' '7.9-7.7 ' '6.8-6.5 ' '5.8-5.5 ' '5.3-5.2 

In [309]:
df_consumption['Fuel Consumption (l/100 km)'] = df_consumption['Combined fuel consumption (WLTP)']
df_consumption['Fuel Consumption (l/100 km)'] = df_consumption['Fuel Consumption (l/100 km)'].fillna(df_consumption['Fuel consumption (economy) - combined'])
df_consumption['Fuel Consumption (l/100 km)'] = df_consumption['Fuel Consumption (l/100 km)'].fillna(df_consumption['Fuel consumption (economy) - combined (NEDC, WLTP equivalent)'])
df_consumption['Fuel Consumption (l/100 km)'] = df_consumption['Fuel Consumption (l/100 km)'].fillna(df_consumption['Fuel consumption (economy) - combined (NEDC)'])
df_consumption = df_consumption.drop(columns=['Combined fuel consumption (WLTP)','Fuel consumption (economy) - combined','Fuel consumption (economy) - combined (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (NEDC)'])

In [310]:
df_consumption

,Fuel Consumption (l/100 km)
0,7.8-8.5
1,5.9-6.4
2,7-7.8
3,6.6-7.3
4,1.5-2
...,...
733,NaN
734,8.4
735,NaN
736,8.3


In [311]:
df_consumption[['min','max']] = df_consumption['Fuel Consumption (l/100 km)'].str.split("-", 1, expand=True)

In [312]:
df_consumption

,Fuel Consumption (l/100 km),min,max
0,7.8-8.5,7.8,8.5
1,5.9-6.4,5.9,6.4
2,7-7.8,7,7.8
3,6.6-7.3,6.6,7.3
4,1.5-2,1.5,2
...,...,...,...
733,NaN,NaN,NaN
734,8.4,8.4,NaN
735,NaN,NaN,NaN
736,8.3,8.3,NaN


In [313]:
df_consumption['max'] = df_consumption['max'].str.strip()
df_consumption = df_consumption.replace(r'^\s*$', np.nan, regex=True)
df_consumption['max'] = df_consumption['max'].fillna(df_consumption['min'])
df_consumption['max'] = pd.to_numeric(df_consumption['max'])
df_consumption['min'] = pd.to_numeric(df_consumption['min'])
df_consumption['Fuel Consumption (l/100 km)'] = df_consumption.mean(axis=1)
df_consumption

C:\Users\nizar\AppData\Local\Temp\ipykernel_26288\2673142846.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_consumption['Fuel Consumption (l/100 km)'] = df_consumption.mean(axis=1)


,Fuel Consumption (l/100 km),min,max
0,8.15,7.8,8.5
1,6.15,5.9,6.4
2,7.40,7.0,7.8
3,6.95,6.6,7.3
4,1.75,1.5,2.0
...,...,...,...
733,NaN,NaN,NaN
734,8.40,8.4,8.4
735,NaN,NaN,NaN
736,8.30,8.3,8.3


In [314]:
df_consumption = df_consumption.drop(columns=['min','max'])

In [315]:
# add fuel consumption data to dropped dataframe and drop old consumption data

df_dropped['Fuel Consumption (l/100 km)'] = df_consumption['Fuel Consumption (l/100 km)']
df_dropped = df_dropped.drop(columns=['Combined fuel consumption (WLTP) ','Fuel consumption (economy) - combined','Fuel consumption (economy) - combined (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (NEDC)' ])


In [316]:
df_dropped

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,Max speed (electric),Recuperation output,System power,System torque,"Average Energy consumption (NEDC, WLTP equivalent)","All-electric range (NEDC, WLTP equivalent)",All-electric range (NEDC),Acceleration 0 - 60 mph,CO2 Emissions (g/km),Fuel Consumption (l/100 km)
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.5,8.15
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.0,6.15
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,7.40
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,6.95
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,140 km/h\r\n\t\t\t\t \r\n\t\t\t\t86.99 mph,20 kW,292 Hp,420 Nm\r\n\t\t\t\t309.78 lb.-ft.,NaN,NaN,NaN,NaN,39.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,733,BMW,3 Series,3 Series (E21),318i (105 Hp) Automatic,1979 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,734,BMW,3 Series,3 Series (E21),318i (105 Hp),1979 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.40
735,735,BMW,3 Series,3 Series (E21),318 (98 Hp),1975 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736,736,BMW,3 Series,3 Series (E21),316 (90 Hp),1975 year,Internal Combustion engine,Sedan,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.30


In [317]:
# Continue cleaning 
# Acceleration 0 - 100 km/h - drop sec from data and to column Name
# Maximum speed - km/h
# Weight-to-power ratio- kg/hp drop and after
# Weight-to-torque ratio - kg/nm
# Battery voltage - V
# Electric motor power - hp 
# Power combustion - hp
# Power per litre - hp/l
# Torque - Nm
# Engine displacement - cm3
# Kerb Weight - kg
# Combined
#     All-electric range (WLTP)- km average
#     All-electric range (NEDC, WLTP equivalent)
#     All-electric range (NEDC)
# Electric motor Torque - nm
# Gross battery capacity - kwh
# Max speed (electric) - k/m
# Recuperation output - kw
# System Power - Combined Power help
# System Torque - Combined Torque Nm

# drop
#     Average Energy consumption (WLTP) 
#     Average Energy consumption (NEDC, WLTP equivalent) 
#     Acceleration 0 - 60 mph 



In [318]:
df_cleaned = df_dropped

In [319]:
df_cleaned.columns

Index(['Unnamed: 0', 'Brand', 'Model ', 'Generation  ',
       'Modification (Engine) ', 'Start of production ',
       'Powertrain Architecture ', 'Body type', 'Seats ', 'Doors ',
       'Fuel Type ', 'Acceleration 0 - 100 km/h', 'Maximum speed ',
       'Emission standard ', 'Weight-to-power ratio ',
       'Weight-to-torque ratio ', 'Battery voltage ', 'Battery location ',
       'Electric motor power ', 'Power ', 'Power per litre ', 'Torque ',
       'Engine displacement ', 'Number of cylinders ', 'Engine aspiration ',
       'Kerb Weight ', 'Drag coefficient (Cd) ', 'Drive wheel ',
       'Battery technology ', 'Electric motor location ',
       'All-electric range (WLTP)', 'Average Energy consumption (WLTP) ',
       'Electric motor Torque ', 'Gross battery capacity ',
       'Max speed (electric) ', 'Recuperation output ', 'System power ',
       'System torque ', 'Average Energy consumption (NEDC, WLTP equivalent) ',
       'All-electric range (NEDC, WLTP equivalent)',
       '

In [320]:
df_cleaned['Acceleration 0 - 100 km/h']= df_cleaned['Acceleration 0 - 100 km/h'].str.replace('[^0-9.]', '', regex=True)

In [321]:
df_cleaned['Acceleration 0 - 100 km/h']

0       4.4
1       4.6
2       5.7
3       5.9
4       5.8
       ... 
733     NaN
734    11.5
735    11.9
736    12.5
737    14.8
Name: Acceleration 0 - 100 km/h, Length: 738, dtype: object

In [322]:
df_cleaned.rename(columns={"Acceleration 0 - 100 km/h": "Acceleration 0 - 100 km/h (s)"}, inplace=True)

In [323]:
df_cleaned.columns

Index(['Unnamed: 0', 'Brand', 'Model ', 'Generation  ',
       'Modification (Engine) ', 'Start of production ',
       'Powertrain Architecture ', 'Body type', 'Seats ', 'Doors ',
       'Fuel Type ', 'Acceleration 0 - 100 km/h (s)', 'Maximum speed ',
       'Emission standard ', 'Weight-to-power ratio ',
       'Weight-to-torque ratio ', 'Battery voltage ', 'Battery location ',
       'Electric motor power ', 'Power ', 'Power per litre ', 'Torque ',
       'Engine displacement ', 'Number of cylinders ', 'Engine aspiration ',
       'Kerb Weight ', 'Drag coefficient (Cd) ', 'Drive wheel ',
       'Battery technology ', 'Electric motor location ',
       'All-electric range (WLTP)', 'Average Energy consumption (WLTP) ',
       'Electric motor Torque ', 'Gross battery capacity ',
       'Max speed (electric) ', 'Recuperation output ', 'System power ',
       'System torque ', 'Average Energy consumption (NEDC, WLTP equivalent) ',
       'All-electric range (NEDC, WLTP equivalent)',
    

In [324]:
df_cleaned['Maximum speed ']= df_cleaned['Maximum speed '].str.replace('[^0-9.]', '', regex=True)
df_cleaned.rename(columns={"Maximum speed ": "Maximum speed (km/hr)"}, inplace=True)

In [325]:
df_cleaned['Weight-to-power ratio '] = df_cleaned['Weight-to-power ratio '].str.split(' k').str[0]
df_cleaned.rename(columns={"Weight-to-power ratio ": "Weight-to-power ratio (kg/hp)"}, inplace=True)

In [326]:
df_cleaned['Weight-to-torque ratio '] = df_cleaned['Weight-to-torque ratio '].str.split(' k').str[0]
df_cleaned.rename(columns={"Weight-to-torque ratio ": "Weight-to-torque ratio (kg/Nm)"}, inplace=True)

In [328]:
df_cleaned['Battery voltage ']= df_cleaned['Battery voltage '].str.replace('[^0-9.]', '', regex=True)
df_cleaned.rename(columns={'Battery voltage ': "Battery Voltage (V)"}, inplace=True)


In [332]:
df_cleaned['Electric motor power '] = df_cleaned['Electric motor power '].str.split(' H').str[0]
df_cleaned.rename(columns={"Electric motor power  ": "Electric motor power (hp)"}, inplace=True)

In [333]:
df_cleaned['Power '] = df_cleaned['Power '].str.split(' H').str[0]
df_cleaned.rename(columns={"Power ": "Combustion Power (hp)"}, inplace=True)

In [334]:
df_cleaned['Power per litre '] = df_cleaned['Power per litre '].str.split(' H').str[0]
df_cleaned.rename(columns={"Power per litre ": "Power per litre (hp/l)"}, inplace=True)

In [335]:
df_cleaned['Torque '] = df_cleaned['Torque '].str.split(' N').str[0]
df_cleaned.rename(columns={"Torque ": "Combustion Torque (Nm)"}, inplace=True)

In [336]:
df_cleaned['Engine displacement '] = df_cleaned['Engine displacement '].str.split(' c').str[0]
df_cleaned.rename(columns={"Engine displacement ": "Engine displacement (cm3)"}, inplace=True)

In [337]:
df_cleaned['Kerb Weight '] = df_cleaned['Kerb Weight '].str.split(' k').str[0]
df_cleaned.rename(columns={"Kerb Weight ": "Kerb Weight (kg)"}, inplace=True)

In [338]:
df_cleaned['Electric motor Torque '] = df_cleaned['Electric motor Torque '].str.split(' N').str[0]
df_cleaned.rename(columns={"Electric motor Torque ": "Electric motor Torque (Nm)"}, inplace=True)

In [339]:
df_cleaned['Gross battery capacity '] = df_cleaned['Gross battery capacity '].str.split(' k').str[0]
df_cleaned.rename(columns={"Gross battery capacity ": "Gross battery capacity (kWh)"}, inplace=True)

In [340]:
df_cleaned['Max speed (electric) ']= df_cleaned['Max speed (electric) '].str.replace('[^0-9.]', '', regex=True)
df_cleaned.rename(columns={"Max speed (electric) ": "Max speed (electric) (km/hr)"}, inplace=True)

In [341]:
df_cleaned['Recuperation output '] = df_cleaned['Recuperation output '].str.split(' k').str[0]
df_cleaned.rename(columns={"Recuperation output ": "Recuperation output (kW)"}, inplace=True)

In [342]:
df_cleaned['System power '] = df_cleaned['System power '].str.split(' H').str[0]
df_cleaned.rename(columns={"System power ": "Total power (hp)"}, inplace=True)

In [343]:
df_cleaned['System torque '] = df_cleaned['System torque '].str.split(' N').str[0]
df_cleaned.rename(columns={"System torque ": "Total torque (Nm)"}, inplace=True)

In [344]:
df_range = pd.DataFrame({'All-electric range (WLTP)': df_cleaned['All-electric range (WLTP)'], 'All-electric range (NEDC, WLTP equivalent)' : df_cleaned['All-electric range (NEDC, WLTP equivalent)'],
                                 'All-electric range (NEDC)' : df_cleaned['All-electric range (NEDC)']})
df_range['All-electric range (WLTP)'] = df_range['All-electric range (WLTP)'].str.split('k').str[0]
df_range['All-electric range (NEDC, WLTP equivalent)'] = df_range['All-electric range (NEDC, WLTP equivalent)'].str.split('k').str[0]
df_range['All-electric range (NEDC)'] = df_range['All-electric range (NEDC)'].str.split('k').str[0]

df_range['All-electric range (km)'] = df_range['All-electric range (WLTP)']
df_range['All-electric range (km)'] = df_range['All-electric range (km)'].fillna(df_range['All-electric range (NEDC, WLTP equivalent)'])
df_range['All-electric range (km)'] = df_range['All-electric range (km)'].fillna(df_range['All-electric range (NEDC)'])

df_range = df_range.drop(columns=['All-electric range (WLTP)','All-electric range (NEDC, WLTP equivalent)','All-electric range (NEDC)'])

df_range[['min','max']] = df_range['All-electric range (km)'].str.split("-", 1, expand=True)
df_range['max'] = df_range['max'].str.strip()
df_range = df_range.replace(r'^\s*$', np.nan, regex=True)
df_range['max'] = df_range['max'].fillna(df_range['min'])
df_range['max'] = pd.to_numeric(df_range['max'])
df_range['min'] = pd.to_numeric(df_range['min'])
df_range['All-electric range (km)'] = df_range.mean(axis=1)
df_range

C:\Users\nizar\AppData\Local\Temp\ipykernel_26288\2252623788.py:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_range['All-electric range (km)'] = df_range.mean(axis=1)


,All-electric range (km),min,max
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,56.0,52.0,60.0
...,...,...,...
733,NaN,NaN,NaN
734,NaN,NaN,NaN
735,NaN,NaN,NaN
736,NaN,NaN,NaN


In [345]:
df_range = df_range.drop(columns=['min','max'])

In [346]:
df_cleaned['All-electric range (km)'] = df_range['All-electric range (km)']

In [347]:
df_cleaned = df_cleaned.drop(columns=['All-electric range (WLTP)','All-electric range (NEDC, WLTP equivalent)','All-electric range (NEDC)', 'Average Energy consumption (WLTP) ','Average Energy consumption (NEDC, WLTP equivalent) ','Acceleration 0 - 60 mph ' ])

In [348]:
df_cleaned.head()

,Unnamed: 0,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,...,Electric motor location,Electric motor Torque (Nm),Gross battery capacity (kWh),Max speed (electric) (km/hr),Recuperation output (kW),Total power (hp),Total torque (Nm),CO2 Emissions (g/km),Fuel Consumption (l/100 km),All-electric range (km)
0,0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.5,8.15,NaN
1,1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,...,Integrated into the transmission,NaN,NaN,NaN,NaN,NaN,NaN,161.0,6.15,NaN
2,2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,7.40,NaN
3,3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,6.95,NaN
4,4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,...,Between the combustion engine and the transmis...,265,12,14086.99,20,292,420,39.0,1.75,56.0


In [349]:
df_cleaned.columns

Index(['Unnamed: 0', 'Brand', 'Model ', 'Generation  ',
       'Modification (Engine) ', 'Start of production ',
       'Powertrain Architecture ', 'Body type', 'Seats ', 'Doors ',
       'Fuel Type ', 'Acceleration 0 - 100 km/h (s)', 'Maximum speed (km/hr)',
       'Emission standard ', 'Weight-to-power ratio (kg/hp)',
       'Weight-to-torque ratio (kg/Nm)', 'Battery Voltage (V)',
       'Battery location ', 'Electric motor power ', 'Combustion Power (hp)',
       'Power per litre (hp/l)', 'Combustion Torque (Nm)',
       'Engine displacement (cm3)', 'Number of cylinders ',
       'Engine aspiration ', 'Kerb Weight (kg)', 'Drag coefficient (Cd) ',
       'Drive wheel ', 'Battery technology ', 'Electric motor location ',
       'Electric motor Torque (Nm)', 'Gross battery capacity (kWh)',
       'Max speed (electric) (km/hr)', 'Recuperation output (kW)',
       'Total power (hp)', 'Total torque (Nm)', 'CO2 Emissions (g/km)',
       'Fuel Consumption (l/100 km)', 'All-electric range (k

In [350]:
#Removed space at end of some column names
df_cleaned.rename(columns={'Model ' :'Model', 'Generation  ': 'Generation',
       'Modification (Engine) ':'Modification (Engine)', 'Start of production ':'Start of production',
       'Powertrain Architecture ':'Powertrain Architecture', 'Seats ':'Seats', 'Doors ':'Doors',
       'Fuel Type ':'Fuel Type','Emission standard ':'Emission standard',
       'Battery location ':'Battery location', 'Electric motor power ':'Electric motor power (hp)', 'Number of cylinders ':'Number of cylinders',
       'Engine aspiration ':'Engine aspiration', 'Drag coefficient (Cd) ':'Drag coefficient (Cd)',
       'Drive wheel ':'Drive wheel', 'Battery technology ':'Battery technology', 'Electric motor location ':'Electric motor location',
       'Average Energy consumption (NEDC) ':'Average Energy consumption (NEDC)'}, inplace=True)

In [351]:
df_cleaned.drop(columns=['Unnamed: 0'], inplace=True)

In [352]:
df_cleaned

,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,Fuel Type,...,Electric motor location,Electric motor Torque (Nm),Gross battery capacity (kWh),Max speed (electric) (km/hr),Recuperation output (kW),Total power (hp),Total torque (Nm),CO2 Emissions (g/km),Fuel Consumption (l/100 km),All-electric range (km)
0,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340i (374 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.5,8.15,NaN
1,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",M340d (340 Hp) Mild Hybrid xDrive Steptronic,"July, 2022 year","MHEV (Mild Hybrid Electric Vehicle, power-assi...",Sedan,5,4,Diesel,...,Integrated into the transmission,NaN,NaN,NaN,NaN,NaN,NaN,161.0,6.15,NaN
2,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) xDrive Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,7.40,NaN
3,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330i (245 Hp) Steptronic,"July, 2022 year",Internal Combustion engine,Sedan,5,4,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,6.95,NaN
4,BMW,3 Series,"3 Series Sedan (G20 LCI, facelift 2022)",330e (292 Hp) xDrive Steptronic,"July, 2022 year",PHEV (Plug-in Hybrid Electric Vehicle),Sedan,5,4,petrol / electricity,...,Between the combustion engine and the transmis...,265,12,14086.99,20,292,420,39.0,1.75,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,BMW,3 Series,3 Series (E21),318i (105 Hp) Automatic,1979 year,Internal Combustion engine,Sedan,5,2,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,BMW,3 Series,3 Series (E21),318i (105 Hp),1979 year,Internal Combustion engine,Sedan,5,2,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.40,NaN
735,BMW,3 Series,3 Series (E21),318 (98 Hp),1975 year,Internal Combustion engine,Sedan,5,2,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736,BMW,3 Series,3 Series (E21),316 (90 Hp),1975 year,Internal Combustion engine,Sedan,5,2,Petrol (Gasoline),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.30,NaN


In [ ]:
df_cleaned.to_csv('BMW_3_Series_Cleaned.csv', index=False)

In [ ]:
df_cleaned = pd.read_csv('BMW_3_Series_Cleaned.csv')